# Семинар 9. Работа с текстами

![Картинка](https://res.cloudinary.com/practicaldev/image/fetch/s--pRQGS79D--/c_limit%2Cf_auto%2Cfl_progressive%2Cq_auto%2Cw_880/https://dev-to-uploads.s3.amazonaws.com/i/c2eanc5wieohaa3scthc.png)

### Примеры задач автоматической обработки текстов:

- классификация текстов

    - анализ тональности (например, позитивный/негативный отзыв)
    - фильтрация спама
    - по теме или жанру

- машинный перевод

- распознавание речи

- извлечение информации

    - именованные сущности (например, извлечение имен, локаций, названий организаций)
    - извлечение фактов и событий

- кластеризация текстов

- оптическое распознавание символов

- проверка правописания

- вопросно-ответные системы

- суммаризация текстов

- генерация текстов

### Классические методы для работы с текстами:

- токенизация

- лемматизация / стемминг

- удаление стоп-слов и пунктуации

- векторное представление текстов (bag of words и TF-IDF)

## Токенизация

Токенизировать — значит, поделить текст на слова, или *токены*.

Самый наивный способ токенизировать текст — разделить с помощью `split`. Но `split` упускает очень много всего, например, не отделяет пунктуацию от слов. Кроме этого, есть ещё много менее тривиальных проблем. Поэтому лучше использовать готовые токенизаторы.

In [ ]:
#!pip install nltk

In [ ]:
from nltk.tokenize import word_tokenize
import numpy as np 
import pandas as pd

In [ ]:
example = 'Но не каждый хочет что-то исправлять:('

In [ ]:
# c помощью split()
example.split()

In [ ]:
# c помощью токенизатора
word_tokenize(example)

В nltk вообще есть довольно много токенизаторов:

In [ ]:
from nltk import tokenize
dir(tokenize)[:16]

Можно получить индексы начала и конца каждого токена:

In [ ]:
wh_tok = tokenize.WhitespaceTokenizer()
list(wh_tok.span_tokenize(example))

Некторые токенизаторы ведут себя специфично:

In [ ]:
tokenize.TreebankWordTokenizer().tokenize("don't stop me")

Для некоторых задач это может быть полезно.

А некоторые предназначены вообще не для текста на естественном языке:

In [ ]:
tokenize.SExprTokenizer().tokenize("(a (b c)) d e (f)")

Есть токенизатор, который может быть полезен для работы с твитами или сообщениями из соц. сетей.
Он сохранит смайлики, хештеги и т.п.

In [ ]:
from nltk.tokenize import TweetTokenizer

tw = TweetTokenizer()
tw.tokenize(example)

_Что почитать:_

- http://mlexplained.com/2019/11/06/a-deep-dive-into-the-wonderful-world-of-preprocessing-in-nlp/
- https://blog.floydhub.com/tokenization-nlp/

## Стоп-слова и пунктуация

*Стоп-слова* — это слова, которые часто встречаются практически в любом тексте и ничего интересного не говорят о конретном документе, то есть играют роль шума. Поэтому их принято убирать. По той же причине убирают и пунктуацию.

In [ ]:
import nltk
#nltk.download("stopwords")

In [ ]:
from nltk.corpus import stopwords
print(stopwords.words('russian'))

In [ ]:
from string import punctuation
punctuation

In [ ]:
noise = stopwords.words('russian') + list(punctuation)

## Лемматизация

[**Лемматизация**](https://en.wikipedia.org/wiki/Lemmatisation) — процесс приведения слова к его нормальной форме (**лемме**):
- для существительных — именительный падеж, единственное число;
- для прилагательных — именительный падеж, единственное число, мужской род;
- для глаголов, причастий, деепричастий — глагол в инфинитиве.


Например, токены «пью», «пил», «пьет» перейдут в «пить». Почему это хорошо?
* Во-первых, мы хотим рассматривать как отдельный признак каждое *слово*, а не каждую его отдельную форму.
* Во-вторых, некоторые стоп-слова стоят только в начальной форме, и без лемматизации выкидываем мы только её.

Для русского есть два хороших лемматизатора: `mystem` и `pymorphy`.

### [Mystem](https://tech.yandex.ru/mystem/)
Как с ним работать:
* можно скачать mystem и запускать [из терминала с разными параметрами](https://tech.yandex.ru/mystem/doc/)
* [pymystem3](https://pythonhosted.org/pymystem3/pymystem3.html) - обертка для питона, работает медленнее, но это удобно

In [ ]:
#!pip install pymystem3

In [ ]:
from pymystem3 import Mystem
import pymystem3
mystem_analyzer = Mystem()

Мы инициализировали Mystem c дефолтными параметрами. А вообще параметры есть такие:
* mystem_bin — путь к `mystem`, если их несколько
* grammar_info — нужна ли грамматическая информация или только леммы (по умолчанию нужна)
* disambiguation — нужно ли снятие [омонимии](https://ru.wikipedia.org/wiki/%D0%9E%D0%BC%D0%BE%D0%BD%D0%B8%D0%BC%D1%8B) - дизамбигуация (по умолчанию нужна)
* entire_input — нужно ли сохранять в выводе все (пробелы, например), или можно выкинуть (по умолчанию оставляется все)

Методы Mystem принимают строку, токенизатор вшит внутри. Можно, конечно, и пословно анализировать, но тогда он не сможет учитывать контекст.

Можно просто лемматизировать текст:

In [ ]:
print(mystem_analyzer.lemmatize(example))

### [Pymorphy](http://pymorphy2.readthedocs.io/en/latest/)
Это модуль на питоне, довольно быстрый и с кучей функций.

In [ ]:
#!pip install pymorphy2
#!pip install pymorphy2-dicts
#!pip install DAWG-Python

In [ ]:
from pymorphy2 import MorphAnalyzer
pymorphy2_analyzer = MorphAnalyzer()

pymorphy2 работает с отдельными словами. Если дать ему на вход предложение — он его просто не лемматизирует, т.к. не понимает.

Метод MorphAnalyzer.parse() принимает слово  и возвращает все возможные его разборы.

У каждого разбора есть тег.
Тег - это набор граммем, характеризующих данное слово. Например, тег 'VERB,perf,intr plur,past,indc' означает, что слово - глагол (VERB) совершенного вида (perf), непереходный (intr), множественного числа (plur), прошедшего времени (past), изъявительного наклонения (indc).


In [ ]:
ana = pymorphy2_analyzer.parse('хочет')
ana

In [ ]:
ana[0].normal_form

### mystem vs. pymorphy

1) *Надеемся, что вы пользуетесь линуксом или маком* — mystem работает невероятно медленно под windows на больших текстах

2) *Снятие омонимии*. Mystem умеет снимать омонимию по контексту (хотя не всегда преуспевает), pymorphy2 берет на вход одно слово и соответственно вообще не умеет дизамбигуировать по контексту.

In [ ]:
homonym1 = 'За время обучения я прослушал больше сорока курсов.'
homonym2 = 'Сорока своровала блестящее украшение со стола.'

# корректно определелил части речи
# NUM - числительное
# S — существительное
print(mystem_analyzer.analyze(homonym1)[-5])
print(mystem_analyzer.analyze(homonym2)[0])

## Стемминг

В отличие от лемматизации, при применении стемминга у всех слов отбрасываются аффиксы (окончания и суффиксы), что необязательно приводит слова к формам, существующим в рассматриваемом языке.

[**Snowball**](http://snowball.tartarus.org/) – фрэймворк для написания алгоритмов стемминга. Алгоритмы стемминга отличаются для разных языков и используют знания о конкретном языке – списки окончаний для разных чистей речи, разных склонений и т.д. 

In [ ]:
from nltk.stem.snowball import SnowballStemmer

In [ ]:
tokenized_example = word_tokenize(example)

In [ ]:
stemmer = SnowballStemmer('russian')
stemmed_example = [stemmer.stem(w) for w in tokenized_example]
print(' '.join(stemmed_example))

Для английского получится что-то такое.

In [ ]:
text = "In my younger and more vulnerable years my father gave me some advice that I've been turning over in my mind ever since.\n\"Whenever you feel like criticizing any one,\" he told me, \"just remember that all the people in this world haven't had the advantages that you've had.\""
print(text)
text_tokenized = [w for w in word_tokenize(text) if w.isalpha()]
print('==========')
print(text_tokenized)

In [ ]:
stemmer = SnowballStemmer('english')
text_stemmed = [stemmer.stem(w) for w in text_tokenized]
print(' '.join(text_stemmed))

_Что почитать:_

- https://en.wikipedia.org/wiki/Stemming
- https://en.wikipedia.org/wiki/Lemmatisation
- https://www.datacamp.com/community/tutorials/stemming-lemmatization-python

## Bag-of-words и TF-IDF

Но как же все-таки работать с текстами, используя стандартные методы машинного обучения? Нужна выборка!


### Bag-of-words

Самый очевидный способ формирования признакового описания текстов — векторизация. Пусть у нас имеется коллекция текстов $D = \{d_i\}_{i=1}^l$ и словарь всех слов, встречающихся в выборке $V = \{v_j\}_{j=1}^d.$ В этом случае некоторый текст $d_i$ описывается вектором $(x_{ij})_{j=1}^d,$ где
$$x_{ij} = \sum_{v \in d_i} [v = v_j].$$

Таким образом, текст $d_i$ описывается вектором количества вхождений каждого слова из словаря в данный текст.

In [ ]:
texts = ['I like my cat.', 'My cat is the most perfect cat.', 'is this cat or is this bread?']

In [ ]:
texts_tokenized = [' '.join([w for w in word_tokenize(t) if w.isalpha()]) for t in texts]
texts_tokenized

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cnt_vec = CountVectorizer()
X = cnt_vec.fit_transform(texts_tokenized)

In [ ]:
cnt_vec.get_feature_names()

In [ ]:
X

In [ ]:
X.toarray()

In [ ]:
pd.DataFrame(X.toarray(), columns=cnt_vec.get_feature_names())

### TF-IDF

Заметим, что если слово часто встречается в одном тексте, но почти не встречается в других, то оно получает для данного текста большой вес, ровно так же, как и слова, которые часто встречаются в каждом тексте. Для того, чтобы разделять эти такие слова, можно использовать статистическую меру TF-IDF, характеризующую важность слова для конкретного текста. Для каждого слова из текста $d$ рассчитаем относительную частоту встречаемости в нем (Term Frequency):
$$
\text{TF}(t, d) = \frac{C(t | d)}{\sum\limits_{k \in d}C(k | d)},
$$
где $C(t | d)$ - число вхождений слова $t$ в текст $d$.

Также для каждого слова из текста $d$ рассчитаем обратную частоту встречаемости в корпусе текстов $D$ (Inverse Document Frequency):
$$
\text{IDF}(t, D) = \log\left(\frac{|D|}{|\{d_i \in D \mid t \in d_i\}|}\right)
$$
Логарифмирование здесь проводится с целью уменьшить масштаб весов, ибо зачастую в корпусах присутствует очень много текстов.

В итоге каждому слову $t$ из текста $d$ теперь можно присвоить вес
$$
\text{TF-IDF}(t, d, D) = \text{TF}(t, d) \times \text{IDF}(t, D)
$$
Интерпретировать формулу выше несложно: действительно, чем чаще данное слово встречается в данном тексте и чем реже в остальных, тем важнее оно для этого текста.

Отметим, что в качестве TF и IDF можно использовать другие [определения](https://en.wikipedia.org/wiki/Tf%E2%80%93idf#Definition).

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vec = TfidfVectorizer()
X = tfidf_vec.fit_transform(texts_tokenized)

In [ ]:
tfidf_vec.get_feature_names()

In [ ]:
pd.DataFrame(X.toarray(), columns=tfidf_vec.get_feature_names())

**Вопросик:** Что изменилось по сравнению с использованием метода `CountVectorizer`?

_Что почитать:_

- https://en.wikipedia.org/wiki/Tf%E2%80%93idf
- https://programminghistorian.org/en/lessons/analyzing-documents-with-tfidf

## Baseline: классификация необработанных n-грамм


In [ ]:
from sklearn.linear_model import LogisticRegression # можно заменить на любимый классификатор
from sklearn.feature_extraction.text import CountVectorizer

Что такое n-граммы:

In [ ]:
from nltk import ngrams

In [ ]:
sent = 'Если б мне платили каждый раз'.split()
list(ngrams(sent, 1)) # униграммы

In [ ]:
list(ngrams(sent, 2)) # биграммы

In [ ]:
list(ngrams(sent, 3)) # триграммы

In [ ]:
list(ngrams(sent, 5)) # ... пентаграммы?

## Задача: классификация твитов по тональности

У нас есть датасет из твитов, про каждый указано, как он эмоционально окрашен: положительно или отрицательно. Задача: предсказывать эмоциональную окраску.

Классификацию по тональности используют, например, в рекомендательных системах, чтобы понять, понравилось ли людям кафе, кино, etc.

Скачиваем куски датасета ([источник](http://study.mokoron.com/)): [положительные](https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv?dl=0), [отрицательные](https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv).

In [ ]:
!wget https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv
!wget https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv

In [ ]:
import pandas as pd
import numpy as np

from sklearn.metrics import classification_report, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import MaxAbsScaler

In [ ]:
# считываем данные и заполняем общий датасет
positive = pd.read_csv('positive.csv', sep=';', usecols=[3], names=['text'])
positive['label'] = 'positive'
negative = pd.read_csv('negative.csv', sep=';', usecols=[3], names=['text'])
negative['label'] = 'negative'
df = positive.append(negative)

In [ ]:
df.shape

In [ ]:
from tqdm import tqdm
stemmer_ru = nltk.stem.snowball.RussianStemmer()

def stem_text(text, stemmer):
    tokens = text.split()
    stemmed_tokens = [stemmer.stem(w) for w in tokens]
    return ' '.join(stemmed_tokens)

stemmed_texts = []
for t in tqdm(df.text[1:10]):
    stemmed_texts.append(stem_text(t, stemmer_ru))

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(df.text, df.label, random_state=13)

Как уже обсуждали, самый простой способ извлечь признаки из текстовых данных — векторизаторы: `CountVectorizer` и `TfidfVectorizer`

Объект `CountVectorizer` делает простую вещь:
* строит для каждого документа (каждой пришедшей ему строки) вектор размерности `n`, где `n` -- количество слов или n-грам во всём корпусе
* заполняет каждый i-тый элемент количеством вхождений слова в данный документ

In [ ]:
pipe_count_vec = make_pipeline(CountVectorizer(ngram_range=(1, 1)),
                                MaxAbsScaler(), 
                              LogisticRegression(max_iter=200, random_state=42))

ngram_range отвечает за то, какие n-граммы мы используем в качестве фичей:<br/>
ngram_range=(1, 1) — униграммы<br/>
ngram_range=(3, 3) — триграммы<br/>
ngram_range=(1, 3) — униграммы, биграммы и триграммы.

В `vec.vocabulary_` лежит словарь: мэппинг слов к их индексам:

In [ ]:
pipe_count_vec.fit(x_train, y_train)

In [ ]:
pred = pipe_count_vec.predict(x_test)
print(classification_report(y_test, pred))

In [ ]:
#pipe_count_vec['countvectorizer'].vocabulary_

Попробуем сделать то же самое для триграмм.

In [ ]:
pipe_count_vec_3 = make_pipeline(CountVectorizer(ngram_range=(1, 1)),
                                MaxAbsScaler(), 
                              LogisticRegression(max_iter=200, random_state=42))

In [ ]:
pipe_count_vec_3.fit(x_train, y_train)

In [ ]:
pred_3 = pipe_count_vec_3.predict(x_test)
print(classification_report(y_test, pred_3))

А теперь для TF-IDF

In [ ]:
pipe_count_vec = make_pipeline(TfidfVectorizer(ngram_range=(1, 1)),
                              LogisticRegression(max_iter=400, random_state=42))

In [ ]:
pipe_count_vec.fit(x_train, y_train)

In [ ]:
pred_tfidf = pipe_count_vec.predict(x_test)
print(classification_report(y_test, pred_tfidf))

## О важности эксплоративного анализа

Но иногда пунктуация бывает и не шумом — главное отталкиваться от задачи. Что будет если вообще не убирать пунктуацию?

In [ ]:
pipe_punct = make_pipeline(CountVectorizer(ngram_range=(1, 1),  tokenizer=word_tokenize),
                                MaxAbsScaler(), 
                              LogisticRegression(max_iter=200, random_state=42))

In [ ]:
pipe_punct.fit(x_train, y_train)
pred_punct = pipe_punct.predict(x_test)
print(classification_report(y_test, pred_punct))

Стоило оставить пунктуацию — и внезапно все метрики устремились к 1. Как это получилось? Среди неё были очень значимые токены (как вы думаете, какие?). Найдите признаки с самыми большими коэффициентами:

Посмотрим, как один из супер-значительных токенов справится с классификацией безо всякого машинного обучения:

In [ ]:
cool_token = 
pred = ['positive' if cool_token in tweet else 'negative' for tweet in x_test]
print(classification_report(y_test, pred))

In [ ]:
np.random.seed(42)

tweets_with_cool_token = [tweet for tweet in x_train if cool_token in tweet]
for tweet in np.random.choice(tweets_with_cool_token, size=10, replace=False):
    print(tweet)

## Символьные n-граммы

Теперь в качестве признаков используем, например, униграммы символов:

In [ ]:
pipe_symbols = make_pipeline(CountVectorizer(ngram_range=(1, 1), analyzer='char'),
                                MaxAbsScaler(), 
                              LogisticRegression(max_iter=200, random_state=42))

In [ ]:
#pipe_symbols['countvectorizer'].get_feature_names()

In [ ]:
pipe_symbols.fit(x_train, y_train)
pred_symbols = pipe_punct.predict(x_test)
print(classification_report(y_test, pred_symbols))

В общем-то, теперь уже понятно, почему на этих данных здесь 1. Так или иначе, на символах классифицировать тоже можно: для некоторых задач (например, для определения языка) признаки-символьные n-граммы могут внести серьезный вклад в качество модели.

Ещё одна замечательная особенность признаков-символов: токенизация и лемматизация не нужна, можно использовать такой подход для языков, у которых нет готовых анализаторов.

_Что почитать:_

- https://web.stanford.edu/~jurafsky/slp3/3.pdf
- https://books.google.com/ngrams

## Регулярные выражения

https://ru.wikipedia.org/wiki/Регулярные_выражения

Вообще, часто бывает так, что для конкретного случая нужен особый способ токенизации, и надо самостоятельно написать регулярное выражение. Или, например, перед работой с текстом, надо почистить его от своеобразного мусора: упоминаний пользователей, url и так далее.

Навык полезный, давайте в нём тоже потренируемся.

In [ ]:
import re

### findall
возвращает список всех найденных совпадений

- ? : ноль или одно повторение
- \* : ноль или более повторений
- \+ : одно или более повторений
- . : любой символ

In [ ]:
result = re.findall('ab+c.', 'abcdefghijkabcabcxabc') 
print(result)

Вопрос на внимательность: почему нет abcx?

In [ ]:
re.findall('ab+c.', 'abbbca')

### split
разделяет строку по заданному шаблону


In [ ]:
result = re.split(',', 'itsy, bitsy, teenie, weenie') 
print(result)

можно указать максимальное количество разбиений

In [ ]:
result = re.split(',', 'itsy, bitsy, teenie, weenie', maxsplit=2) 
print(result)

### sub
ищет шаблон в строке и заменяет все совпадения на указанную подстроку

параметры: (pattern, repl, string)

In [ ]:
result = re.sub('a', 'b', 'abcabc')
print(result)

### compile
компилирует регулярное выражение в отдельный объект

In [ ]:
# Пример: построение списка всех слов строки:
prog = re.compile('[А-Яа-яё\-]+')
prog.findall("Слова? Да, больше, ещё больше слов! Что-то ещё.")

**Задание**: вернуть список доменов (@gmail.com) из списка адресов электронной почты:

```
abc.test@gmail.com, xyz@test.in, test.first@analyticsvidhya.com, first.test@rest.biz
```

In [ ]:
emails = 'abc.test@gmail.com, xyz@test.in, test.first@analyticsvidhya.com, first.test@rest.biz'

In [ ]:
# your code here

_Что почитать:_

- https://habr.com/ru/post/115825/
- https://www.regular-expressions.info/
- https://regexr.com/

### JUST FOR FUN

In [ ]:
#!conda install -c conda-forge wordcloud -y
import wordcloud as wc
import matplotlib.pyplot as plt
import numpy as np

from PIL import Image
from sklearn.datasets import fetch_20newsgroups

In [ ]:
data = fetch_20newsgroups(subset='all', categories=['sci.med'], remove=('headers', 'footers', 'quotes'))

In [ ]:
mask = np.array(Image.open("sherlock.png"))

In [ ]:
wc_im = wc.WordCloud(width = 3000, 
                     height = 2000, 
                     random_state=1, 
                     background_color='white', 
                     contour_color='steelblue',
                     contour_width=3,
                     collocations=False,
                     mask=mask,
                     stopwords = stopwords.words('english')).generate(data['data'][0])

In [ ]:
plt.figure(figsize=(10, 12))
plt.imshow(wc_im) 
plt.axis("off")
plt.show()